In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs

%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return


In [2]:
def optionslam_scrape(ticker):
    site = 'https://www.optionslam.com/earnings/stocks/' + ticker
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    soup = soup.prettify()
    earnings_dict = {'Ticker': ticker}
    
    # Check if there's weekly options
    curr7_implied = "Current 7 Day Implied Movement:"
    implied_move_weekly = "Implied Move Weekly:"
    nextearnings = "Next Earnings Date:"
    if curr7_implied not in soup:
        return False
    
    # Parsing if weekly options exist
    # Next earnings date and before or after
    earnings_start_string = "Next Earnings Date:"
    earnings_end_string = '</font>'
    raw_earnings_string = (soup.split(earnings_start_string))[1].split(earnings_end_string)[0].replace('\n','').strip()
    earnings_date = str((raw_earnings_string.split('<b>'))[1].split('<font size="-1">')).split("'")[1].strip()
    earnings_time = str(raw_earnings_string[-2:].strip()).strip()
    
    earnings_dict['Date'] = earnings_date
    earnings_dict['Earnings Time'] = earnings_time
    
    # Parsing 7 day implied move if weekly option exists
    ending_string = '<font size="-2">'
    curr_7 = (soup.split(curr7_implied))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    earnings_dict['Current 7 Day Implied'] = curr_7
    
    # Parsing Weekly Implied move if weekly option exists
    if implied_move_weekly in soup:
        weekly_implied = (soup.split(implied_move_weekly))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    else:
        weekly_implied = ''
    earnings_dict["Implied Move Weekly"] = weekly_implied
    
    return earnings_dict

In [3]:
check_list = list(pd.read_csv('checklist.csv')['Tickers'])

df = {'Ticker':"", 
      'Date':'', 
      'Earnings Time':'', 
      'Current 7 Day Implied':'', 
      'Implied Move Weekly':''}
df = pd.DataFrame(df, index = [])

i = 0
for ticker in check_list:
    temp_name = optionslam_scrape(ticker)
    if temp_name == False:
        continue
    else:
        temp_df = pd.DataFrame(temp_name, index = [0])
        df = pd.concat([df, temp_df], axis = 0)
        i += 1
        

In [4]:
df[['Ticker','Date','Earnings Time','Current 7 Day Implied','Implied Move Weekly']].to_csv('nov20list.csv')

In [6]:
df

,Current 7 Day Implied,Date,Earnings Time,Implied Move Weekly,Ticker
0,4.84%,"Nov. 21, 2017",AC,7.57%,HPE
0,4.06%,"Nov. 21, 2017",BO,,DLTR
0,5.71%,"Nov. 21, 2017",AC,10.36%,GME
0,3.05%,"Nov. 21, 2017",AC,,HPQ
0,3.33%,"Nov. 21, 2017",BO,5.27%,LOW
0,1.91%,"Nov. 21, 2017",AC,,MDT
0,2.40%,"Nov. 21, 2017",AC,5.76%,CRM
0,2.57%,"Nov. 22, 2017",BO,5.77%,DE
